In [91]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
import pickle
from dotenv import load_dotenv
import os
#from langchain.tools import DuckDuckGoSearchTool
from langchain.agents import Tool
from langchain.tools import BaseTool
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [92]:
llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')

In [117]:
def resume_question(input=""):
    with open("graphic_designer.pkl", "rb") as f:
        docsearch = pickle.load(f)
    chain = load_qa_chain(llm=llm, chain_type="stuff")
    docs = docsearch.similarity_search(input)
    answer=chain.run(input_documents=docs, question=input)
    return answer

resume_tool = Tool(
    name='resume_question',
    func= resume_question,
    description="Useful for when you need to answer questions about the resumes and the candidates. input is the question"
)

#search = DuckDuckGoSearchTool()

def jobsite_question(input="",jobsite=""):
    with open("jobsite_embeddings.pkl", "rb") as f:
        jobsite = pickle.load(f)
    chain = load_qa_chain(llm=llm, chain_type="stuff")
    docs = jobsite.similarity_search(input)
    answer=chain.run(input_documents=docs, question=input)
    return answer

job_tool = Tool(
    name='job_question',
    func= jobsite_question,
    description="Useful for when you need to answer questions about the job requirements .input is the question"
)

In [118]:
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

tools =[resume_tool,job_tool]

memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [119]:
from langchain.tools import DuckDuckGoSearchRun


def duck(input=""):
    search = DuckDuckGoSearchRun()
    answer=search.run(input)
    return answer


search_tool = Tool(
    name='search_question',
    func= duck,
    description="Useful for when you need to answer questions that are unknown from the internet. input is the question"
)


In [120]:
tools =[resume_tool,job_tool]

memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [121]:
conversational_agent = initialize_agent(
    agent='zero-shot-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate'
)

In [126]:
conversational_agent("Is the candidate suitable for the job,  give only a yes and no answer. A maybe means a no?")



> Entering new AgentExecutor chain...
I need to evaluate the candidate's qualifications and compare them to the job requirements.
Action: resume_question
Action Input: "What are the candidate's qualifications and experience?"
Observation: The candidate has a Bachelor of Fine Arts in Graphic Design from the University of North Carolina at Charlotte, with a GPA of 3.75/4.0. They have experience as a Graphic Design Intern at Wray Ward, where they created various digital ads to help increase connection with students on Facebook, Instagram & Twitter, developed images for the main website utilizing Adobe Illustrator and Photoshop, and researched brand standards of the university. They also have experience as a Graphic Design Intern and Gallery Assistant at UNC Charlotte, where they assisted gallery staff at art shows & events, maintained social media for the gallery, and greeted visitors to the gallery and provided information regarding the current exhibits. Additionally, they have skills 

{'input': 'Is the candidate suitable for the job,  give only a yes and no answer. A maybe means a no?',
 'output': "Based on the candidate's qualifications and experience, they do not seem suitable for the Operations Expert role at Apple as they do not have experience managing inventory or leading a team."}